In [14]:
import json
from pathlib import Path

import pandas as pd

OG = Path('/Volumes/krauss/Lisa/GMR/Array/SpecialBox/Cuthbert/250723/data/metadata_manifest.json')
# CSV = Path('/Volumes/krauss/Lisa/GMR/Array/SpecialBox/Cuthbert/250723/results_FULL/communicating_results.csv')
CSV = Path('/Volumes/krauss/Lisa/GMR/Array/SpecialBox/Cuthbert/250723/results_FULL/communicating_results_new.csv')
CSV_NEW = Path('/Volumes/krauss/Lisa/GMR/Array/SpecialBox/Cuthbert/250723/results_FULL/communicating_results_new.csv')
JSONL = Path('/Volumes/krauss/Lisa/GMR/Array/SpecialBox/Cuthbert/250723/results_FULL/processing_results.jsonl')

print(OG)
print(OG.is_file())
print(CSV)
print(CSV.is_file())
print(JSONL)
print(JSONL.is_file())

/Volumes/krauss/Lisa/GMR/Array/SpecialBox/Cuthbert/250723/data/metadata_manifest.json
True
/Volumes/krauss/Lisa/GMR/Array/SpecialBox/Cuthbert/250723/results_FULL/communicating_results_new.csv
True
/Volumes/krauss/Lisa/GMR/Array/SpecialBox/Cuthbert/250723/results_FULL/processing_results.jsonl
True


In [15]:
column_to_search = 'ROI-label'

text_to_find = '002A_N'

try:
    df = pd.read_csv(CSV)

    matching_rows = df[df[column_to_search] == text_to_find]

    count = len(matching_rows)

    print(f"Found {count} rows where '{column_to_search}' is '{text_to_find}'.")

    total_rows = len(df)

    unique_entries_count = df[column_to_search].nunique()

    print(f"Total number of rows in the file: {total_rows}")
    print(f"Number of unique entries in column '{column_to_search}': {unique_entries_count}")

    line_count = 0
    with JSONL.open('r') as f:
        for line in f:
            line_count += 1

    print(f"The file '{JSONL}' contains {line_count} lines.")


except FileNotFoundError:
    print(f"Error: The file '{CSV}' was not found.")
except KeyError:
    print(f"Error: The column '{column_to_search}' was not found in the CSV file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Found 1225 rows where 'ROI-label' is '002A_N'.
Total number of rows in the file: 788900
Number of unique entries in column 'ROI-label': 644
The file '/Volumes/krauss/Lisa/GMR/Array/SpecialBox/Cuthbert/250723/results_FULL/processing_results.jsonl' contains 175 lines.


In [13]:
"""
Combines data from the data manifest and the results manifest to create a CSV.
"""
try:
    with JSONL.open("r") as f:
        print(f"Parsing '{JSONL.name}' as JSON Lines.")
        results_data = []
        for line in f:
            if line.strip():
                results_data.append(json.loads(line))

    results_map = {
        entry.get("image_filename"): entry 
        for entry in results_data if entry.get("image_filename")
    }
except Exception as e:
    print(f"Could not load or parse results manifest: {e}", exc_info=True)

print(f"Parsing {len(results_data)} entries to create CSV...")
records = []

for entry in results_data:

    filename = entry.get("image_filename")
    result_entry = results_map.get(filename)

    if not result_entry:
        print(f"No matching result found for processed image: {filename}. Skipping entry.")
        continue

    timestamp = result_entry.get("image_timestamp")
    image_analysis = result_entry.get("image_analysis")

    headers = []
    if not headers:
        for item in image_analysis:
            if "ROI-label" in item:
                headers = list(item.keys())
                break
    
    for analysis_item in image_analysis:
        if "ROI-label" in analysis_item:
            record = {"timestamp": timestamp}
            for field in headers:
                value = analysis_item.get(field)
                if isinstance(value, dict):
                    value = value.get("Median")
                record[field] = value
                
                records.append(record)

if not records:
    print("No valid records found to generate a CSV file. Skipping.")

# Create the DataFrame
df = pd.DataFrame(records)
try:
    df.to_csv(CSV_NEW, index=False)
    print(f"Successfully saved CSV to {CSV_NEW} (under lock).")
except Exception as e:
    print(f"Failed to save CSV file under lock at {CSV_NEW}: {e}", exc_info=True)

Parsing 'processing_results.jsonl' as JSON Lines.
Parsing 175 entries to create CSV...
Successfully saved CSV to /Volumes/krauss/Lisa/GMR/Array/SpecialBox/Cuthbert/250723/results_FULL/communicating_results_new.csv (under lock).
